In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, powerlaw
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import Galactocentric, galactocentric_frame_defaults, ICRS, SkyCoord, Galactic
%run Convert-Coordinates.ipynb

In [2]:
# CONSTANTS -> REMOVE AFTER TESTING
U0 = 11.1 
V0 = 12.24
W0 = 7.25
Theta_0 = 220
galcen_params = dict(galcen_v_sun=np.array([U0, V0+Theta_0, W0]) * u.km/u.s, galcen_distance=8*u.kpc)

**convert_vel_to_pm**

**Inputs**: Right ascencion and declination (given in equatorial system, spherical coordinates), distance from sun (pc), radial, tangential, and z velocities (galactocentric system, cylindrical coordinates) 

**Output**: Proper motion and radial velocity (equatorial system)

In [4]:
# ra and dec given in equatorial coords. Velocities in galactocentric coords in cylindrical system
def convert_vel_to_pm(ra, dec, distance, v_R, v_phi, v_z):
    star_eq_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=distance*u.pc, frame=ICRS())
    star_galcen_coord = star_eq_coord.transform_to(Galactocentric(**galcen_params))
    phi = star_galcen_coord.cylindrical.phi
    # v_x, v_y, v_z = f(v_R, v_phi, f_z, phi)
    # Original
    # v_x = v_R*np.cos(phi) - v_phi*np.sin(phi)
    # v_y = v_R*np.sin(phi) + v_phi*np.cos(phi)
  
    # try pi - phi
    v_x = v_R*np.cos(np.pi*u.rad - phi) - v_phi*np.sin(np.pi*u.rad - phi)
    v_y = v_R*np.sin(np.pi*u.rad - phi) + v_phi*np.cos(np.pi*u.rad - phi)
    # # v_z stays the same
    star_galcen_coord_vel = Galactocentric(x=star_galcen_coord.x, y=star_galcen_coord.y, z=star_galcen_coord.z, v_x=v_x*u.km/u.s, v_y=v_y*u.km/u.s, v_z=v_z*u.km/u.s, **galcen_params)
    # print(star_galcen_coord_vel)
    star_eq_coord_vel = star_galcen_coord_vel.transform_to(ICRS())
    return star_eq_coord_vel

In [5]:
# Cartesian to Cylindrical Velocities
def v_xyz_to_cyl(x, y, z, v_x, v_y, v_z):
    v_r = (x*v_x + y*v_y) / np.sqrt(x**2 + y**2)
    v_phi = (-y*v_x + x*v_y) / np.sqrt(x**2 + y**2)
    print("Sun's cylindrical velocities: ", v_r, v_phi, v_z)
    return v_r, v_phi, v_z

# Cylindrical to Cartesian Velocities
def v_cyl_to_xyz(v_r, v_phi, v_z, phi):
    v_x = -v_r*np.cos(phi) - v_phi*np.sin(phi)
    v_y = v_r*np.sin(phi) - v_phi*np.cos(phi)
    print("Sun's xyz velocities: ", v_x, v_y, v_z)
    return v_x, v_y, v_z

In [6]:
def find_pm(pm_ra_cosdec, pm_dec):
    return np.sqrt(pm_ra_cosdec**2 + pm_dec**2)

In [7]:
# CODE TO TEST CONVERSION FUNCTIONS
# converted sun's velocities to cylindrical velocities
# say starting coordinate is (10, 10, 10) = (x,y,z)
x = 10 * u.parsec
y = 10 * u.parsec
z = 10 * u.parsec
# Use instead of arctan so sign info gets preserved, also takes care of case that x=0
phi = np.arctan2(y, x)
x = x.to(u.km)
y = y.to(u.km)
z = z.to(u.km)


v_x_sun = U0 * u.km/u.second
v_y_sun = (V0 + Theta_0) * u.km/u.second
v_z_sun = W0 * u.km/u.second
  
# v_r_sun, v_phi_sun, v_z_sun = v_xyz_to_cyl(x, y, z, v_x_sun, v_y_sun, v_z_sun)


# v_x1, v_y1, v_z1 = v_cyl_to_xyz(v_r_sun, v_phi_sun, v_z_sun, phi)
# print("Check by converting back to xyz: ", v_x1, v_y1, v_z1)
# print("Angle is: ", np.rad2deg(np.arctan(y/x)))

# ra, dec, phi, distance, v_R, v_phi, v_z
star3 = convert_vel_to_pm(ra=0, dec=0, distance=0, v_R=16.5, v_phi=0.8061, v_z=7.25)
print("star3", star3)

star3 <ICRS Coordinate: (ra, dec, distance) in (deg, deg, pc)
    (86.42366563, 26.52040218, 1.01843357e-12)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (2.49731125e+16, -4.09291358e+16, 2.96582227)>


In [8]:
# galactic coordinate system is centered around sun
galactic_coord = SkyCoord(l=0*u.deg, b=90*u.deg, frame="galactic")

equatorial_coord = galactic_coord.icrs
eq_ra = equatorial_coord.ra.deg
eq_dec = equatorial_coord.dec.deg
# print(eq_ra, eq_dec)
# STAR MOVING PARALLEL TO SUN
star4 = convert_vel_to_pm(ra=eq_ra, dec=eq_dec, distance=100, v_R=U0, v_phi=V0+Theta_0, v_z=W0)
print(star4)
print(star4.transform_to(Galactic()))

<ICRS Coordinate: (ra, dec, distance) in (deg, deg, pc)
    (192.85947789, 27.12825241, 100.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (7.31617892e-06, -5.25106727e-06, 1.10811271e-08)>
<Galactic Coordinate: (l, b, distance) in (deg, deg, pc)
    (49.04448557, 90., 100.)
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-7.07522113e-06, -5.57148341e-06, 1.10811271e-08)>


In [9]:
print((star4.pm_dec * 100*u.pc / u.rad).to(u.km/u.s))

-2.489252928592177e-06 km / s


In [27]:
# Do v_phi=0
galactic_coord = SkyCoord(l=0*u.deg, b=0*u.deg, frame="galactic")

equatorial_coord = galactic_coord.icrs
eq_ra = equatorial_coord.ra.deg
eq_dec = equatorial_coord.dec.deg
star5 = convert_vel_to_pm(ra=eq_ra, dec=eq_dec, distance=100, v_R=U0, v_phi=0, v_z=W0)
print(star5)
print(star5.transform_to(Galactic()))

<ICRS Coordinate: (ra, dec, distance) in (deg, deg, pc)
    (266.40498829, -28.93617776, 100.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-255.25778018, -418.15603675, 0.00021603)>
<Galactic Coordinate: (l, b, distance) in (deg, deg, pc)
    (0., 5.08888749e-14, 100.)
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-489.90918078, 0.0007196, 0.00021603)>
